In [2]:
import cv2
import mediapipe as mp
import numpy as np
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler
import math as m
import pandas as pd
from tensorflow.keras.models import load_model
from sklearn.preprocessing import StandardScaler

In [3]:
mp_drawing = mp.solutions.drawing_utils
mp_pose = mp.solutions.pose

In [4]:
def calculate_angle(a,b,c):
    a = np.array(a) # First
    b = np.array(b) # Mid
    c = np.array(c) # End
    
    radians = np.arctan2(c[1]-b[1], c[0]-b[0]) - np.arctan2(a[1]-b[1], a[0]-b[0])
    angle = np.abs(radians*180.0/np.pi)
    
    if angle >180.0:
        angle = 360-angle
        
    return angle 

In [5]:
def Angle(x1, y1, x2, y2, x3, y3):
    
    if x1 == x2 or x2 == x3 :
        degree = 90
    elif y1 == y2 or y2 == y3:
        degree = 0
    else :
        theta = m.atan( (y1- y2) / ( x1- x2)) - m.atan( (y3 - y2) / (x3 - x2))
        degree = int(180/m.pi)*theta

    
    if degree <0 :
        degree = degree*(-1)
    return degree

In [6]:
def KneeDistance(x1, y1, x2, y2):
    
    dist = round(m.sqrt((x2-x1)**2+(y2-y1)**2),4)
    
    return dist

In [7]:
a = []
b = []
c = []
d = []
e = []

In [9]:
def squat_TR():
    a = []
    b = []
    c = []
    d = []
    e = []

    cap = cv2.VideoCapture(0)

    # Curl counter variables
    counter = 0 

    stage = 'down'
    ## Setup mediapipe instance
    with mp_pose.Pose(min_detection_confidence=0.5, min_tracking_confidence=0.5) as pose:
        while cap.isOpened():

            ret, frame = cap.read()
            
            # Recolor image to RGB
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            image.flags.writeable = False
        
            # Make detection
            results = pose.process(image)
        
            # Recolor back to BGR
            image.flags.writeable = True
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            
            # Extract landmarks
            try:
                landmarks = results.pose_landmarks.landmark
                
                # Get coordinates
                hip = [landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y]
                knee = [landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y]
                ankle = [landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y]
                # Calculate angle
                angle = calculate_angle(hip, knee, ankle)
                
                leftKneeAngle = Angle(landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_ANKLE.value].y)
                
                leftHipAngle = Angle(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].x,landmarks[mp_pose.PoseLandmark.LEFT_HIP.value].y,
                    landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.LEFT_SHOULDER.value].y)
                
                rightKneeAngle = Angle(landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_ANKLE.value].y)
                
                rightHipAngle = Angle(landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_HIP.value].y,
                    landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_SHOULDER.value].y)
                kneeD = KneeDistance(landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.LEFT_KNEE.value].y,
                            landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].x,landmarks[mp_pose.PoseLandmark.RIGHT_KNEE.value].y)

                # Visualize angle
                cv2.putText(image, str(angle), 
                            tuple(np.multiply(knee, [1920, 1080]).astype(int)), 
                            cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 255, 255), 2, cv2.LINE_AA
                                    )
                if counter == 5 :
                    cv2.waitKey(3000)
                    break
                
                if len(a) == 0 or len(a) == len(e) :


                    if angle <170 : 
                        print(1)
                        a.append(leftKneeAngle)
                        a.append(leftHipAngle)
                        a.append(rightKneeAngle)
                        a.append(rightHipAngle)
                        a.append(kneeD)
                
                        
                elif len(b) < len(a) :


                    if angle < 140:
                        print(2)
                        b.append(leftKneeAngle)
                        b.append(leftHipAngle)
                        b.append(rightKneeAngle)
                        b.append(rightHipAngle)
                        b.append(kneeD)
                        
                elif len(c) < len(b) :


                    if angle < 95 :
                        print(3)
                        c.append(leftKneeAngle)
                        c.append(leftHipAngle)
                        c.append(rightKneeAngle)
                        c.append(rightHipAngle)
                        c.append(kneeD)
                        stage = 'up'
                        
                elif len(d) < len(c) :

            
                    if angle >140:
                        print(4)
                        d.append(leftKneeAngle)
                        d.append(leftHipAngle)
                        d.append(rightKneeAngle)
                        d.append(rightHipAngle)
                        d.append(kneeD)
                        
                elif len(e) < len(d) :
        

                    if angle >170 :
                        
                        e.append(leftKneeAngle)
                        e.append(leftHipAngle)
                        e.append(rightKneeAngle)
                        e.append(rightHipAngle)
                        e.append(kneeD)
                        print(5)
                        stage = 'down'
                        
                        counter += 1
                        globals()[f'f{counter}'] = a+b+c+d+e
                        a = []
                        b = []
                        c = []
                        d = []
                        e = []
                
                        
            except:
                pass
            
            # Render curl counter
            # Setup status box
            cv2.rectangle(image, (0,0), (225,73), (245,117,16), -1)
            
            # Rep data
            cv2.putText(image, 'REPS', (15,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, str(counter), 
                        (10,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            # Stage data
            cv2.putText(image, 'STAGE', (65,12), 
                        cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0,0,0), 1, cv2.LINE_AA)
            cv2.putText(image, stage, 
                        (60,60), 
                        cv2.FONT_HERSHEY_SIMPLEX, 2, (255,255,255), 2, cv2.LINE_AA)
            
            
            # Render detections
            mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_pose.POSE_CONNECTIONS,
                                    mp_drawing.DrawingSpec(color=(245,117,66), thickness=2, circle_radius=2), 
                                    mp_drawing.DrawingSpec(color=(245,66,230), thickness=2, circle_radius=2) 
                                    )               
            
            cv2.imshow('Mediapipe Feed', image)

            if cv2.waitKey(10) & 0xFF == ord('q'):
                break

        cap.release()
        cv2.destroyAllWindows()
    df = pd.DataFrame([f1,f2, f3, f4, f5])
    df.columns = ["leftKneeAngle0", "leftHipAngle0", "rightKneeAngle0", "rightHipAngle0", "KneeDistance0",
                "leftKneeAngle5", "leftHipAngle5", "rightKneeAngle5", "rightHipAngle5", "KneeDistance5",
                "leftKneeAngle8", "leftHipAngle8", "rightKneeAngle8", "rightHipAngle8", "KneeDistance8",
                "leftKneeAngle12", "leftHipAngle12", "rightKneeAngle12", "rightHipAngle12", "KneeDistance12",
                "leftKneeAngle15", "leftHipAngle15", "rightKneeAngle15", "rightHipAngle15", "KneeDistance15"]
    scalar = StandardScaler()
    x_data_scaled = scalar.fit_transform(df)
    model = load_model('bsquatpredict.h5')
    a = np.array(x_data_scaled)
    b = model.predict(a)
    c = np.argmax(b, axis=1)
    val=[]
    acc=0.0
    for i in range(len(c)):
        if c[i]==0:
            acc+=20
            val.append("정자세입니다.")
        elif c[i]==1:
            val.append("척추가 말렸습니다.")
        elif c[i]==2:
            val.append("고개를 내려주세요.")
        elif c[i]==3:
            val.append("무릎을 모아주세요.")

    print(val)
    print(acc)

1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1
2
3
4
5
1/1 [==============================] - 0s 114ms/step
['척추가 말렸습니다.', '무릎을 모아주세요.', '정자세입니다.', '척추가 말렸습니다.', '척추가 말렸습니다.']
20.0


In [10]:
knee

[0.3531087636947632, 1.093076229095459]

In [22]:
df = pd.DataFrame([f1,f2, f3, f4, f5])
df

,0,1,2,3,4,5,6,7,8,9,...,15,16,17,18,19,20,21,22,23,24
0,159.239679,163.050263,166.878472,165.888025,0.0384,132.600345,140.902220,125.450788,135.024812,0.0306,...,143.206480,148.578166,143.142793,146.092379,0.0161,170.463800,168.991688,170.696573,166.953648,0.0319
1,166.252330,168.684377,166.634919,170.184759,0.0357,109.600599,123.276449,97.637027,111.860713,0.0459,...,145.666560,154.226334,143.824877,150.140720,0.0388,177.426490,175.244383,173.437834,174.284107,0.0229
2,147.547358,151.232500,143.726926,149.174323,0.0306,137.260610,141.349051,129.239437,134.414179,0.0213,...,140.355682,143.770240,140.936351,141.960547,0.0155,5.910455,164.868913,171.992393,164.018988,0.0240
3,163.530862,155.954268,164.071976,156.328489,0.0109,128.858557,132.008803,118.905018,120.560893,0.0260,...,141.763290,150.539163,140.967436,145.340381,0.0240,170.415830,166.967542,172.868331,166.835312,0.0264
4,160.853540,159.880556,158.854745,158.127544,0.0314,136.567508,139.997743,127.357599,130.545134,0.0260,...,146.352061,148.083970,140.122351,141.984467,0.0272,173.370343,170.852422,172.475035,169.086865,0.0300


In [23]:
df.columns = ["leftKneeAngle0", "leftHipAngle0", "rightKneeAngle0", "rightHipAngle0", "KneeDistance0",
              "leftKneeAngle5", "leftHipAngle5", "rightKneeAngle5", "rightHipAngle5", "KneeDistance5",
              "leftKneeAngle8", "leftHipAngle8", "rightKneeAngle8", "rightHipAngle8", "KneeDistance8",
              "leftKneeAngle12", "leftHipAngle12", "rightKneeAngle12", "rightHipAngle12", "KneeDistance12",
              "leftKneeAngle15", "leftHipAngle15", "rightKneeAngle15", "rightHipAngle15", "KneeDistance15"]

In [24]:
df

,leftKneeAngle0,leftHipAngle0,rightKneeAngle0,rightHipAngle0,KneeDistance0,leftKneeAngle5,leftHipAngle5,rightKneeAngle5,rightHipAngle5,KneeDistance5,...,leftKneeAngle12,leftHipAngle12,rightKneeAngle12,rightHipAngle12,KneeDistance12,leftKneeAngle15,leftHipAngle15,rightKneeAngle15,rightHipAngle15,KneeDistance15
0,159.239679,163.050263,166.878472,165.888025,0.0384,132.600345,140.902220,125.450788,135.024812,0.0306,...,143.206480,148.578166,143.142793,146.092379,0.0161,170.463800,168.991688,170.696573,166.953648,0.0319
1,166.252330,168.684377,166.634919,170.184759,0.0357,109.600599,123.276449,97.637027,111.860713,0.0459,...,145.666560,154.226334,143.824877,150.140720,0.0388,177.426490,175.244383,173.437834,174.284107,0.0229
2,147.547358,151.232500,143.726926,149.174323,0.0306,137.260610,141.349051,129.239437,134.414179,0.0213,...,140.355682,143.770240,140.936351,141.960547,0.0155,5.910455,164.868913,171.992393,164.018988,0.0240
3,163.530862,155.954268,164.071976,156.328489,0.0109,128.858557,132.008803,118.905018,120.560893,0.0260,...,141.763290,150.539163,140.967436,145.340381,0.0240,170.415830,166.967542,172.868331,166.835312,0.0264
4,160.853540,159.880556,158.854745,158.127544,0.0314,136.567508,139.997743,127.357599,130.545134,0.0260,...,146.352061,148.083970,140.122351,141.984467,0.0272,173.370343,170.852422,172.475035,169.086865,0.0300


In [28]:
scalar = StandardScaler()
x_data_scaled = scalar.fit_transform(df)
model = load_model('bsquatpredict.h5')
a = np.array(x_data_scaled)
b = model.predict(a)
c = np.argmax(b, axis=1)
val=[]
for i in range(len(c)):
    if c[i]==0:
        val.append("정자세입니다.")
    elif c[i]==1:
        val.append("척추가 말렸습니다.")
    elif c[i]==2:
        val.append("고개를 내려주세요.")
    elif c[i]==3:
        val.append("무릎을 모아주세요.")

In [29]:
val

['무릎을 모아주세요.', '척추가 말렸습니다.', '척추가 말렸습니다.', '무릎을 모아주세요.', '정자세입니다.']